# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Weather data file
weather_output_data = "output_data/cities.csv"

# Read the weather data csv file
weather_data = pd.read_csv(weather_output_data)
weather_data.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.1927,168.8643,43.45,83,82,6.04,NZ,1595624400
1,1,Tautira,-17.7333,-149.1500,76.50,74,16,16.58,PF,1595624400
2,2,Pevek,69.7008,170.3133,45.59,71,89,7.61,RU,1595624400
3,3,Vryheid,-27.7695,30.7917,48.25,90,19,6.55,ZA,1595624400
4,4,Ribeira Grande,38.5167,-28.7000,72.00,67,10,7.56,PT,1595624400


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)


In [5]:
# Plot Heatmap

locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Develop criteria to locate an ideal location based on the weather
temp = weather_data["Max Temp"].astype(float)
wind_speed = weather_data["Wind Speed"].astype(float)

temp_factor = (temp < 80) & (temp > 70)
wind_factor = wind_speed < 10
cloudiness_factor = weather_data.Cloudiness == 0
final_criteria = temp_factor & wind_factor & cloudiness_factor 

# Filter the weather_data_df dataframe
ideal_weather = weather_data[final_criteria]
ideal_weather = ideal_weather.dropna()
ideal_weather = ideal_weather.reset_index()
del ideal_weather["index"]
ideal_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,219,Amantea,39.1332,16.0746,77.83,68,0,2.17,IT,1595624400
1,252,Guarujá,-23.9931,-46.2564,75.74,78,0,2.37,BR,1595624400
2,289,Pindamonhangaba,-22.9239,-45.4617,74.55,41,0,2.19,BR,1595624400
3,396,Aksu,41.1231,80.2644,78.69,27,0,2.04,CN,1595624400
4,429,Qaryat Sulūq,31.6682,20.2521,75.92,72,0,9.93,LY,1595624400


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_weather

# Created a column for Hotel Name 
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,219,Amantea,39.1332,16.0746,77.83,68,0,2.17,IT,1595624400,
1,252,Guarujá,-23.9931,-46.2564,75.74,78,0,2.37,BR,1595624400,
2,289,Pindamonhangaba,-22.9239,-45.4617,74.55,41,0,2.19,BR,1595624400,
3,396,Aksu,41.1231,80.2644,78.69,27,0,2.04,CN,1595624400,
4,429,Qaryat Sulūq,31.6682,20.2521,75.92,72,0,9.93,LY,1595624400,


In [9]:
# Create hotels list
hotels = []

# Set up parameters dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the heatmap_df to locate airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    request_hotel = requests.get(base_url, params=params)
    
    # convert to json
    hotel_address = request_hotel.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotels.append(hotel_address["results"][0]["name"])
      
    except (KeyError, IndexError):
        hotels.append("Closest hotel not found")

        # Dataframe with nearest hotel
ideal_weather["Closest Hotel"] = hotels
ideal_weather


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Closest Hotel
0,219,Amantea,39.1332,16.0746,77.83,68,0,2.17,IT,1595624400,,Mediterraneo Palace Hotel
1,252,Guarujá,-23.9931,-46.2564,75.74,78,0,2.37,BR,1595624400,,Casa Grande Hotel Resort & Spa
2,289,Pindamonhangaba,-22.9239,-45.4617,74.55,41,0,2.19,BR,1595624400,,colonial plaza
3,396,Aksu,41.1231,80.2644,78.69,27,0,2.04,CN,1595624400,,Pudong Holiday Hotel
4,429,Qaryat Sulūq,31.6682,20.2521,75.92,72,0,9.93,LY,1595624400,,Closest hotel not found
5,452,Şuhut,38.5311,30.5458,70.11,35,0,4.70,TR,1595624400,,Şuhut Belediye Oteli
6,457,Ürümqi,43.8010,87.6005,77.92,26,0,3.04,CN,1595624400,,JinJiang International Hotel Urumqi


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Closest Hotel: {hotel}" for hotel in hotels])
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))